# Chapter 21 - Code and Exercises - Notebook \#1
Code, exercises, and experiments from chapter 21 of "Advances in Financial Machine Learning" by Marcos López de Prado (2018).

In [29]:
# Imports.
import time
import numpy as np
import pandas as pd

from itertools import combinations_with_replacement, product

from mlfinlab.brute_force.combinatorial_opt import *
from mlfinlab.brute_force.comb_opt_example import *

---
## 21.5 AN INTERGER OPTIMIZATION APPROACH
Reframing the asset allocation problem by discretizing the funds available. Consider the number of ways one can allocate $K$ units of capital in $N$ assets, where we assume $K>N$. Essentially we want to find the number of of non-negative integer solutions (though zero is still seems to be a possibility) to $x_1+...+x_N=K$, which has a combinatorial solution $\binom{K+N-1}{N-1}$.

While this bears similarity to a classic integer partitioning problem, note here that order *is* important, since the assets to which capital is allocated is encoded by the order of the permutation. For example, if $K=6$, then partitions $(1,2,3)$ and $(3,2,1)$ are very different ways of allocating units of capital between assets $(x_1, x_2, x_3)$.

Code snippets 21.1, 21.2, and 21.3 are implemented in `mlfinlab.brute_force.combinatorial_opt`, which are algorithms to generate the set of all partitions, $p^{N,K} = \{ \{ p_i \}_{i=1,...,N} | p_i \in \mathbb{W}, \sum_{i=1}^{N} p_i = K \}$, where $\mathbb{W}$ are the set of natural numbers including zero (whole numbers).

---
## 21.6 A NUMERICAL EXAMPLE
The code snippets below are used to generate matrices of random values for use in the portfolio optimization problem described in chapter 21. The code written above is used to solve for the dynamically optimal portfolio.

In [83]:
# SNIPPET 21.4. PRODUCE A RANDOM MATRIX OF A GIVEN RANK
# =====================================================
def random_matrix_with_rank(n_samples, n_cols, rank, sigma=0, hom_noise=True):
    """
    Produce a random 'n_samples'-by-'n_cols' matrix 'X' with given rank,
    and apply a specified random noise.
    
    :param n_samples: (int) Number of rows to appear in the matrix.
    :param n_cols: (int) Number of column to appear in the matrix.
    :param rank: (int) The rank of the matrix.
    :param sigma: (float) The variance of the noise to be produced.
    :param hom_noise: (bool) Whether or not to add homoscedastic noise.
     If False, the added noise is heterscedastic.
    :return: (numpy.array) The array 'X'.
    """
    rng = np.random.RandomState()
    U, _, _ = np.linalg.svd(rng.randn(n_cols, n_cols))
    X = np.dot(rng.randn(n_samples, rank), U[:, :rank].T)
    if hom_noise:
        X += sigma * rng.randn(n_samples, n_cols)  # Adding homoscedastic noise.
    else:
        sigmas = sigma * (rng.rand(n_cols)+.5)  # Adding heteroscedastic noise.
        X += rng.randn(n_samples, n_cols) * sigmas
    return X

In [85]:
random_matrix_with_rank(2, 3, 2)

array([[-0.39688496,  0.99431002, -0.21017588],
       [ 0.91928988, -1.10951248, -1.32584739]])

[0, 1, 2]

In [33]:
w = 5
w is None

False